# Quantum Pipeline

Define small feature map (angle encoding, ZZFeatureMap).

Define variational ansatz (1–3 layers).

Measure expectation values → feature vector.

Train classical classifier on these features.

In [14]:
import pennylane as qml
import pennylane.numpy as np
import pennylane_lightning
from pennylane.optimize import NesterovMomentumOptimizer

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc
# from joblib import Parallel, delayed
import joblib
from multiprocessing import Pool
import itertools
import time
import pandas as pd
# import numpy as np
import os
from dotenv import load_dotenv

# from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

In [2]:
data = np.load("data/mnist01_pca4.npz")    #from dataprep notebook

X_train, y_train = data["X_train"], data["y_train"]  
X_test, y_test = data["X_test"], data["y_test"]

# x = features/ dimensions --> 4
# y = labels
n_qubits=4

#backened 
# dev= qml.device("default.qubit", wires=n_qubits)
dev = qml.device("lightning.qubit", wires=n_qubits)    #faster simulator, uses compiled c++ 


print("PCA reduced shape xtrain:  ", X_train.shape)
print("PCA reduced shape xtest:  ", X_test.shape)
print("ytrain labels: ", y_train.shape)
print("ytest labels: ",y_test.shape)


PCA reduced shape xtrain:   (12665, 4)
PCA reduced shape xtest:   (2115, 4)
ytrain labels:  (12665,)
ytest labels:  (2115,)


## Quantum Feature Map

Classical data is transformed into quantum states. For the main pipeline, I will be using PennyLane as it is easier to build custom circuits and I can compare feature maps in a single framework. Pennylane also supports Qiskit backeneds for a future plugin to run on IBM devices. 


<br>

This is a quantum map $\phi(\mathbf{x})$ from classical feature vector $\mathbf{x}$ to quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$ in the Hilbert space, achieved through applying the parameterized unitary operator $\mathcal{U}_{\Phi(\mathbf{x})}$ to the initial state $|0\rangle^{n}$, n being the num of qubits for encoding. This is done so quantum algorithms can use the data. 

Unitary operations are a building block of Qcircuits, where a matrix U is unitary if $$U^\dagger U = U U^\dagger = I$$ 

Some examples are the Pauli X Gate or Hadamard Gate, or in this case, the Rotation Gate $R_y(\theta)$. 
$$
R_y(\theta) = \begin{bmatrix}\cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2)\end{bmatrix}, \quad
R_y^\dagger(\theta) = R_y(-\theta), \quad
R_y(\theta) R_y^\dagger(\theta) = I
$$


### ZZ Map Manual - PennyLane

For a ZZ Feature Map, PennyLane does not already have a built in function so we have to do it manually using Hadamard gates, along wiht RZ and ZZ rotations for each qubit pair. This just makes it simpler to integrate over the course of this project. The Qiskit version of this gate is shown in my quantum testing ground. 

Also, out of all encoding methods I could've picked, Angle Encoding seemed to be the most simple and effective when working with circuits (perfect when I am working on a simple project).

The ZZ feature map is a version of the Pauli feature map, which includes entangling gates (ZZ interactions thorugh controlled-phase gates), done after initial data encoding rotations. This is done to allow feature map to capture any potential correlations between features and then maps the data to a more complex hilbert space. 

There are layers of Hadamard gates, single qubit phase rotations, and 2 qubit controlled phase rotations. 

<br>

Hadamard Gates

Creates equal superposition of either state within a qubit

$$
H = \tfrac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\[6pt]
1 & -1
\end{pmatrix}
$$

RZ Gates 

Single qubit phase rotation around Z axis, diagonal gate

$$
R_Z(\phi) 
= \exp\!\left(-i \tfrac{\phi}{2} Z\right) 
= 
\begin{pmatrix}
e^{-i\phi/2} & 0 \\[6pt]
0 & e^{\,i\phi/2}
\end{pmatrix}
$$


CNOT Gates

Flips target qubit if control qubit is 1.  --> encodes pairwise correlations

$$
\text{CNOT} = CX =
\begin{pmatrix}
1 & 0 & 0 & 0 \\[6pt]
0 & 1 & 0 & 0 \\[6pt]
0 & 0 & 0 & 1 \\[6pt]
0 & 0 & 1 & 0
\end{pmatrix}
$$

<br>

ZZ Feature Map

Encodes the pairwise correlations between features (by constructing a unitary transformation)

$$
U_{\mathrm{ZZ}}(x) = \exp\left( i \sum_i x_i Z_i + i \sum_{i<j} x_i x_j Z_i Z_j \right)
$$


In [3]:
sample_vec = X_train[0]      #1st of 12665
print("Example feature vector: ", sample_vec)  


Example feature vector:  [-8.31395886 -6.09814717 -1.37530231 -1.22747553]


In [4]:
def zz_feature_map(x, wires, reps=1):      #(classical data vector, qubits in use, number of layers/repititions)
    n_qubits = len(wires)

    for _ in range(reps):
        for i in wires: 
            qml.Hadamard(wires=i)            #hadamard gates, uniform superposition of states --> gives room for inference in future entanglement 

        for i, wire in enumerate(wires):     #single qubit phase rotations (encodes classical data to qubit phases)
            qml.RZ(2*x[i], wires=wire)       #RZ rotation, angle = 2x[i] 

        for i in range(n_qubits-1):                    #ZZ entangling layer for pairs of neighboring qubits/wires
            qml.CNOT(wires=[wires[i], wires[i+1]])
            qml.RZ(2*x[i]*x[i+1], wires=wires[i+1])    #performs rotation around Z axis by given amount (angle = 2x[i] * x[i+1]) ENCODES PAIRWISE CORRELATIONS BETWEEN FEATURES x[i] and x[i+1]
            qml.CNOT(wires=[wires[i], wires[i+1]])     #uncomputes entanglement (so final effect is ZZ interaction)


# he
@qml.qnode(dev)
def qnode_measure(x):
    zz_feature_map(x, wires=range(4), reps=1)
    return qml.state()


print(f"ZZFeatureMap state vector: \n {qnode_measure(sample_vec)} \n") 
print(qml.draw(qnode_measure, level='device')(sample_vec))    


ZZFeatureMap state vector: 
 [ 0.24383773+0.05516485j  0.10352574+0.22755751j  0.08408267-0.23543599j
  0.17845754-0.17507971j  0.2042117 -0.14421367j  0.23835067+0.07542517j
 -0.1233491 +0.21745114j -0.20592768+0.14175257j -0.2463512 -0.04255689j
 -0.11508695-0.22193466j -0.07186839+0.23944714j -0.16922122+0.18402222j
  0.18491341+0.16824694j -0.02216916+0.24901512j -0.23895882-0.07347573j
 -0.18288099-0.17045394j] 

0: ──H──RZ(-16.63)─╭●─────────────╭●──────────────────────────────────┤ ╭State
1: ──H──RZ(-12.20)─╰X──RZ(101.40)─╰X─╭●────────────╭●─────────────────┤ ├State
2: ──H──RZ(-2.75)────────────────────╰X──RZ(16.77)─╰X─╭●───────────╭●─┤ ├State
3: ──H──RZ(-2.45)─────────────────────────────────────╰X──RZ(3.38)─╰X─┤ ╰State


In this circuit, you can see how each qubit has a Hadamard gate applied (without this, qubits starting in |0⟩ and RZ would only add a global phase - unobservable, as RZ works with relative phase differences between |0⟩ and |1⟩). 

Then, a phase rotation of (x) amount around the Z axis to encode the classical features x[i] as a Z axis phase shift of angle 2x[i].  

Finally, we have the entanglement layers that work through each adjacent pair of qubits to perform phase shifts. Applies controlled phase depending on both qubits and encodes pairwaise correlations into the state. 

**Creates a nonlienar feature map in Hilbert space.**

A CNOT sandwich, which is seen within the multiqubit entanglement layer above, allows for conditional application of a targets rotation, depending on the control qubit. So, in the end, we are maintaining the computational values of the qubit but keeping the phase entanglement of the qubits (it would look the same on the 0,1 level but now hidden quantum phase correlations encode the data)

## Variational Ansatz 

Ansatz: basic architecture of circuit, set of gates that act on specific subsystems with a few assumptions about the appropriate training circuit. Can be generic/problem neutral (hardware ansatz) or can be problem specific. 

Quantum Ansatz: Parameterized Quantum Circuit (PQC) that represents a family of possible quantum stats that are controlled by tunable paramters (normally angles)

$$
|\psi(\theta)\rangle = U(\theta) |0\rangle
$$

ansatz circuit: $$U(\theta)$$ parameters: $$\theta = [\theta_1, \theta_2, \ldots]$$ 


Method used to approimate ground state (lowest energy eingenstate) of quantum system by selecting a trial wavefunction with adjustable parameters. The expectation value of the energy will always be >= to the true ground state energy, so if we optimize the trial wavefunction, we can minimize the expectation value. Built from rotation gates (paramterized by angles) and entangling gates --> these form a layer, we can form many and stakc them to increase expresssiveness. 

**Analogy to CC**

Similar to how a NN layer has weights = rotational angles in the quantum gates. 
- training adjusts the angles to minimize some error/loss 
- patterns to how the gates connect = circuits architecture 

Also similar to deeper NN layers, we can stack them to increase expressiveness


In [5]:
def var_ansatz(params): #params = layers, n_qubits, 3 
    for l in range(params.shape[0]):     #loops through all layers
        for i in range(n_qubits):     #single qubit rotations 
            qml.Rot(params[l,i,0], params[l,i,1], params[l,i,2], wires=i)     # paramertized rotations
        
        for i in range(n_qubits-1):     #entanglers for non linear relationships 
            qml.CNOT(wires=[i, i+1])     #[i:control, i+1:target]


#intial params tensors - random, will be trained in ansatz 
params = np.random.randn(2, n_qubits, 3)     # 2 layers, n_qubits =4, 3 euler angles for each rotation axis 

@qml.qnode(dev)
def ansatz_preview(params):
    var_ansatz(params)
    return qml.state()

print(qml.draw(ansatz_preview)(params))



0: ──Rot(-0.42,0.49,-0.17)─╭●──Rot(0.55,0.25,-1.42)─────────────────────── ···
1: ──Rot(0.67,0.72,0.49)───╰X─╭●─────────────────────Rot(0.38,-0.48,-2.34) ···
2: ──Rot(0.31,1.37,0.11)──────╰X────────────────────╭●──────────────────── ···
3: ──Rot(-0.24,1.06,-0.16)──────────────────────────╰X──────────────────── ···

0: ··· ─╭●───────────────────────────┤  State
1: ··· ─╰X─────────────────────╭●────┤  State
2: ··· ──Rot(-0.34,-0.11,0.81)─╰X─╭●─┤  State
3: ··· ──Rot(-0.66,1.64,-0.00)────╰X─┤  State


This shows rotation gates on each qubit [0..3], with 3 values inside the parenthesis to represent the 3 euler angles used for the rotation (qml.Rot(θ, φ, λ)). The entanglers (CNOT gates) are the vertical lines and dot connecting the qubits. For instance, the dot on qubit 0 indicates that it is the control and the target is qubit 1. There are 6 CNOT gates (3 per layer), with rotations done on each qubit before applying CNOT gates. 

## Measure Expectation Value

In [9]:
@qml.qnode(dev, diff_method="parameter-shift")      #alllows for batches 
def qnode_measure(x, params):
    # x_batch = np.atleast_2d(x_batch)   #ensures shape [batch,featues]       
    zz_feature_map(x, wires=range(4), reps=1)              #encodes data
    var_ansatz(params)       #transforms encoded state w var ansatz
    #returns expectation values  - explicitly, so that joblib can use it in precise order
    # return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]    
    return (
        qml.expval(qml.PauliZ(0)),
        qml.expval(qml.PauliZ(1)),
        qml.expval(qml.PauliZ(2)),
        qml.expval(qml.PauliZ(3)),
    )




def get_quantum_features(X, params):
    feats = np.array([qnode_measure(x,params) for x in X])
    return feats           #shape [N,n_qubits]

In [7]:
print(qml.draw(qnode_measure)(sample_vec, params))      #now displays zzfeature map  and var ansatz

0: ──H──RZ(-16.63)─╭●─────────────╭●──Rot(-0.42,0.49,-0.17)─────────────────────────────────── ···
1: ──H──RZ(-12.20)─╰X──RZ(101.40)─╰X─╭●────────────────────────────────╭●──Rot(0.67,0.72,0.49) ···
2: ──H──RZ(-2.75)────────────────────╰X──────────────────────RZ(16.77)─╰X─╭●────────────────── ···
3: ──H──RZ(-2.45)─────────────────────────────────────────────────────────╰X────────────────── ···

0: ··· ─╭●─────────Rot(0.55,0.25,-1.42)───────────────────────────────────────────────── ···
1: ··· ─╰X─────────────────────────────────────────────────────╭●──Rot(0.38,-0.48,-2.34) ···
2: ··· ───────────╭●─────────────────────Rot(0.31,1.37,0.11)───╰X─╭●──────────────────── ···
3: ··· ──RZ(3.38)─╰X─────────────────────Rot(-0.24,1.06,-0.16)────╰X──────────────────── ···

0: ··· ─╭●───────────────────────────┤  <Z>
1: ··· ─╰X─────────────────────╭●────┤  <Z>
2: ··· ──Rot(-0.34,-0.11,0.81)─╰X─╭●─┤  <Z>
3: ··· ──Rot(-0.66,1.64,-0.00)────╰X─┤  <Z>


Here you can see the original ZZ featuremap applied to the qubits: each one has a Hadamard gate for placing them into superpositions, then an RZ gate to rotate the qubits around the Z axis by an angle proportionate to its xi value (encodes the data into it's phase), then uses CNOT entanglers and RZ gates to connect qubits. 

The second layer is the Variational Ansatz, where the rotation gates (with the 3 angles) allow for any single-qubit rotation in the bloch sphere, which makes it possible for the ansatz to explore all possible states. 

The final measurement for each qubit is <Z>, which is the measured expectation value of PauliZ operator on each qubit. They then become a classical feature vector we input into the classic models. 

### Training Params



This trains the parameters using the variational ansatz and uses a gradient descent optimizer to help train/optimize the params over 20 epochs (using a MSE loss function). 

This block of code runs for several hours, so I have it save the results to the folder /notebooks/params. The bottom of the code (training loop) will be commented out so it will not run each time. 

We use Mean Squared Error (MSE) cost function, which allows us to see if the parameters are a better (closer) fit than randomly assigned parameters. 

In [11]:
# defining a cost function - MSE 
def cost(params, X, y):         #params (layers, nqubits, 3 euler angles), x=classical data [N, nfeatures], y=labels (0 or 1)
    preds = np.array([qnode_measure(x, params) for x in X])   #for each x, runs zzmap, var ansatz, returns expect value of Z for qubit - [N, nfeatures]
    #loss function
    return np.mean((preds[:,0] - y)**2)  # MSE betwen predicted quantum expect values and classic labels y


# threaded parallelism, allows us to avoid device reinitialization every 
# def cost_parallel(params, X, y, n_processes=4):     
#     losses = Parallel(n_jobs=n_processes, backend="threading")(
#         delayed(cost)(params, X_chunk, y_chunk)
#         for X_chunk, y_chunk in zip(np.array_split(X, n_processes), np.array_split(y, n_processes)))
#     return np.mean(losses)


opt = qml.GradientDescentOptimizer(stepsize=0.1)    #tweaks params to minimize cost function
subset = 500  

#training loop
for epoch in range(20):        #optimizes over 50 iterations 
    params = opt.step(lambda p: cost(p, X_train[:subset], y_train[:subset]), params)   #we try to reduce cost while updating params each step 
    # cost_val = cost_parallel(params, X_train[:subset], y_train[:subset])
    cost_val = cost(params, X_train[:subset], y_train[:subset])

    print(f"Epoch: {epoch}, Cost: {cost_val}")


np.save("params/trained_params.npy", params)    #saves trained numpy array of params to params folder
print(qml.draw(ansatz_preview)(params))   #recheck 


Epoch: 0, Cost: 0.5681169368784493
Epoch: 1, Cost: 0.5658078883063472
Epoch: 2, Cost: 0.5636157105087594
Epoch: 3, Cost: 0.5615384178075813
Epoch: 4, Cost: 0.5595734541628717
Epoch: 5, Cost: 0.5577177824399701
Epoch: 6, Cost: 0.555967970558568
Epoch: 7, Cost: 0.5543202729366392
Epoch: 8, Cost: 0.5527707060605613
Epoch: 9, Cost: 0.5513151173963952
Epoch: 10, Cost: 0.549949247195239
Epoch: 11, Cost: 0.5486687830321393
Epoch: 12, Cost: 0.5474694071516742
Epoch: 13, Cost: 0.5463468368757272
Epoch: 14, Cost: 0.5452968584640859
Epoch: 15, Cost: 0.5443153549117183
Epoch: 16, Cost: 0.5433983282239482
Epoch: 17, Cost: 0.5425419167385811
Epoch: 18, Cost: 0.5417424080682868
Epoch: 19, Cost: 0.5409962482227901
0: ──Rot(-0.42,0.13,-0.17)─╭●──Rot(0.55,-0.05,-1.42)─────────────────────── ···
1: ──Rot(0.67,0.71,0.50)───╰X─╭●──────────────────────Rot(0.38,-0.48,-2.34) ···
2: ──Rot(0.31,1.37,0.11)──────╰X─────────────────────╭●──────────────────── ···
3: ──Rot(-0.24,1.06,-0.16)──────────────────────────

## Training Model (Classical Classifiers)

### Logistic Regression

In [12]:
params = np.load("params/trained_params.npy")
#expectation values are treated like classical features 
Xq_train = get_quantum_features(X_train, params)     
Xq_test = get_quantum_features(X_test, params)

q_lg_clf = LogisticRegression().fit(Xq_train, y_train)
q_lg_y_pred = q_lg_clf.predict(Xq_test)  

print("Quantum-feature accuracy:", q_lg_clf.score(Xq_test, y_test))
print("\n\nConfusion Matrix: \n", confusion_matrix(y_test, q_lg_y_pred))    
print("\nClassification Report:\n", classification_report(y_test, q_lg_y_pred))

Quantum-feature accuracy: 0.5366430260047281


Confusion Matrix: 
 [[   0  980]
 [   0 1135]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.54      1.00      0.70      1135

    accuracy                           0.54      2115
   macro avg       0.27      0.50      0.35      2115
weighted avg       0.29      0.54      0.37      2115



c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res

### SVM

In [13]:
q_svm_clf = SVC(kernel='linear', C=1.0, random_state=42)
q_svm_clf.fit(Xq_train, y_train)   

q_svm_y_pred = q_svm_clf.predict(Xq_test)  

print("Quantum-feature accuracy:", q_svm_clf.score(Xq_test, y_test))
print("\n\nConfusion Matrix: \n", confusion_matrix(y_test, q_svm_y_pred))    
print("\nClassification Report:\n", classification_report(y_test, q_svm_y_pred))

Quantum-feature accuracy: 0.5366430260047281


Confusion Matrix: 
 [[   0  980]
 [   0 1135]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.54      1.00      0.70      1135

    accuracy                           0.54      2115
   macro avg       0.27      0.50      0.35      2115
weighted avg       0.29      0.54      0.37      2115



c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res

### Save Models

In [19]:
joblib.dump(q_lg_clf, "../results/quantum_logreg.pkl")
joblib.dump(q_svm_clf, "../results/quantum_svm.pkl")

['../results/quantum_svm.pkl']

## Experiments 


Sweep qubits (2–4), depth, shots, noise.

Log results to CSV.

Include IBM device run if possible (small circuit).

### Load IBM account

In [ ]:
load_dotenv()  
api_key = os.getenv("IBM_API_KEY")
IBMQ.save_account("api_key")      #run only once
IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters=lambda b: b.configuration().n_qubits >= n_qubits and
                                                  not b.configuration().simulator and
                                                  b.status().operational==True))  


print("Using IBM backend:", backend.name())


In [ ]:
params = np.load("params/trained_params.npy")      #trained parameters
 
#trained models
q_lg_clf = joblib.load("models/quantum_logreg.pkl")
q_svm_clf = joblib.load("models/quantum_svm.pkl")   


### Sweep Qubits (Circuit Hyperparameters)

Looping over a range of circuit hyperparamters (number of qubits, depth (# of layers in var. ansatz), shot (# of times qcircuit is sampled), noise (whether to stimulate device noise - will run on IBM)). 

- More qubits would mean a richer feature space but more noise/cost. 
- More depth/layers would mean higher expressivity but is more difficult to train
- More shots means lower measurement noise but a longer runtime
- More noise means noise realism tradeoff


In [ ]:
n_qubit_list = [2,3,4]
depth_list = [1,2]
shot_list= [None, 1024]      #none means no sampling - analytic
noise_list = [False, True]    #true = use simulator 

In [ ]:
lg_results = []
svm_results = []   

for n_qubits, depth, shots, noise in itertools.product(n_qubit_list, depth_list, shot_list, noise_list):
    start = time.time()

    #adjust device configration
    if noise:
        dev =  qml.device("qiskit.ibmq", wires=n_qubits, backend=backend, shots=shots)     #now runs on ibm hardware    
    else:
        dev = qml.device("default.qubit", wires=n_qubits, shots=shots)


    @qml.qnode(dev)
    def qnode_measure(x, params=params):
        zz_feature_map(x, wires=range(n_qubits), reps=depth)
        var_ansatz(params, n_qubits)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    #extract quantum features
    Xq_train = get_quantum_features(X_train, params)
    Xq_test = get_quantum_features(X_test, params)


    #log reg 
    lg_y_pred = q_lg_clf.predict(Xq_test)
    lg_acc = np.mean(q_lg_y_pred == y_test)

    #svm model 
    svm_y_pred = q_svm_clf.predict(Xq_test)   
    svm_acc = np.mean(q_svm_y_pred == y_test)

    lg_results.append({
        "n_qubits": n_qubits, 
        "depth": depth,  
        "shots": shots if shots else "analytic", 
        "noise": noise,
        "accuracy": lg_acc,
        "runtime_sec": round(time.time() - start, 2)
    })    

    svm_results.append({
        "n_qubits": n_qubits,
        "depth": depth,
        "shots": shots if shots else "analytic",
        "noise": noise,   
        "accuracy":svm_acc,    
        "runtime_sec": round(time.time() - start, 2)   
    })

    print(f"Run ({n_qubits}q, depth={depth}, shots={shots}, noise={noise}) → acc={acc:.3f}")

### Save Results

In [ ]:
q_lg_df = pd.DataFrame(lg_results)
q_lg_df.to_csv("results/quantum_log_reg_sweep_results.csv", index=False)
print("\nSaved to results/quantum_log_reg_sweep_results.csv")


q_svm_df = pd.DataFrame(svm_results)
q_svm_df.to_csv("results/quantum_svm_sweep_results.csv", index=False)
print("\nSaved to results/quantum_svm_sweep_results.csv")